# 01 - Project Setup

## Agentic Logistics Control System

This notebook establishes the project infrastructure including:
- Environment configuration
- Dependency installation
- Project structure validation
- Configuration management

### Control Loop Role: Infrastructure

```
OBSERVE -> REASON -> PLAN -> DECIDE -> ACT -> FEEDBACK -> (loop)
```

This notebook sets up the foundation for all control loop components.

## 1. Environment Setup

In [ ]:
# Install dependencies
!pip install langgraph langchain langchain-core openai sympy pydantic pydantic-settings fastapi uvicorn httpx pandas numpy python-dotenv tenacity structlog -q

In [ ]:
# Verify installations
import sys
print(f"Python version: {sys.version}")

# Check core dependencies
dependencies = [
    ("langgraph", "langgraph"),
    ("langchain", "langchain"),
    ("openai", "openai"),
    ("sympy", "sympy"),
    ("pydantic", "pydantic"),
    ("fastapi", "fastapi"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
]

for name, module in dependencies:
    try:
        imported = __import__(module)
        version = getattr(imported, "__version__", "unknown")
        print(f"✓ {name}: {version}")
    except ImportError:
        print(f"✗ {name}: NOT INSTALLED")

## 2. Project Structure Validation

In [ ]:
from pathlib import Path
import os

# Set project root
PROJECT_ROOT = Path(os.getcwd()).parent
print(f"Project root: {PROJECT_ROOT}")

# Expected directories
expected_dirs = [
    "notebooks",
    "src/perception",
    "src/reasoning",
    "src/planning",
    "src/decision",
    "src/action",
    "src/feedback",
    "src/orchestration",
    "src/api",
    "dashboard/src/components",
    "tests",
    "config",
    "data",
]

print("\nValidating project structure:")
for dir_path in expected_dirs:
    full_path = PROJECT_ROOT / dir_path
    exists = full_path.exists()
    status = "✓" if exists else "✗"
    print(f"  {status} {dir_path}")
    
    # Create if missing
    if not exists:
        full_path.mkdir(parents=True, exist_ok=True)
        print(f"    -> Created {dir_path}")

## 3. Configuration Management

In [ ]:
# Add src to Python path
import sys
sys.path.insert(0, str(PROJECT_ROOT / "src"))
sys.path.insert(0, str(PROJECT_ROOT))

# Load configuration
from config.settings import settings, Settings

print("Configuration loaded:")
print(f"  Project Root: {settings.PROJECT_ROOT}")
print(f"  Grok Model: {settings.GROK_MODEL}")
print(f"  Observation Interval: {settings.OBSERVATION_INTERVAL}s")
print(f"  Log Level: {settings.LOG_LEVEL}")
print(f"  Headless Mode: {settings.HEADLESS_MODE}")

In [ ]:
# Create .env template if not exists
env_template = '''# Agentic Logistics Control System Configuration

# xAI/Grok API Configuration
XAI_API_KEY=your-api-key-here
XAI_BASE_URL=https://api.x.ai/v1
GROK_MODEL=grok-beta

# LLM Settings
LLM_TEMPERATURE=0.7
LLM_MAX_TOKENS=4096
LLM_TIMEOUT=60
LLM_MAX_RETRIES=3

# Control Loop Settings
OBSERVATION_INTERVAL=30
MAX_PLANNING_SCENARIOS=5
DECISION_CONFIDENCE_THRESHOLD=0.7

# API Settings
API_HOST=0.0.0.0
API_PORT=8000

# Logging
LOG_LEVEL=INFO

# Production Settings
HEADLESS_MODE=false
CONTINUOUS_LOOP=false
'''

env_path = PROJECT_ROOT / ".env.template"
if not env_path.exists():
    env_path.write_text(env_template)
    print(f"Created .env.template at {env_path}")
else:
    print(f".env.template already exists")

# Check for actual .env file
actual_env = PROJECT_ROOT / ".env"
if not actual_env.exists():
    print("\n⚠️  No .env file found. Copy .env.template to .env and configure your settings.")
else:
    print("✓ .env file found")

## 4. Logging Setup

In [ ]:
import structlog
import logging

def configure_logging(log_level: str = "INFO", json_format: bool = False):
    """Configure structured logging for the application."""
    
    # Configure standard logging
    logging.basicConfig(
        format="%(message)s",
        level=getattr(logging, log_level.upper()),
    )
    
    # Configure structlog
    processors = [
        structlog.contextvars.merge_contextvars,
        structlog.processors.add_log_level,
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.StackInfoRenderer(),
    ]
    
    if json_format:
        processors.append(structlog.processors.JSONRenderer())
    else:
        processors.append(structlog.dev.ConsoleRenderer())
    
    structlog.configure(
        processors=processors,
        wrapper_class=structlog.make_filtering_bound_logger(logging.INFO),
        context_class=dict,
        logger_factory=structlog.PrintLoggerFactory(),
        cache_logger_on_first_use=True,
    )
    
    return structlog.get_logger()

# Initialize logger
logger = configure_logging(settings.LOG_LEVEL)
logger.info("Logging configured", level=settings.LOG_LEVEL)

## 5. Utility Functions

In [ ]:
# Write utility functions to src
utils_code = '''"""Utility functions for the Logistics AI Control System."""
import structlog
import logging
from pathlib import Path
from typing import Any
import json
from datetime import datetime


def get_logger(name: str = __name__) -> structlog.BoundLogger:
    """Get a configured logger instance."""
    return structlog.get_logger(name)


def load_json(path: Path) -> dict[str, Any]:
    """Load JSON from file."""
    with open(path, "r") as f:
        return json.load(f)


def save_json(data: dict[str, Any], path: Path) -> None:
    """Save data to JSON file."""
    with open(path, "w") as f:
        json.dump(data, f, indent=2, default=str)


def timestamp() -> str:
    """Get current ISO timestamp."""
    return datetime.utcnow().isoformat() + "Z"


def ensure_dir(path: Path) -> Path:
    """Ensure directory exists, create if not."""
    path.mkdir(parents=True, exist_ok=True)
    return path
'''

utils_path = PROJECT_ROOT / "src" / "utils.py"
utils_path.write_text(utils_code)
print(f"Created {utils_path}")

In [ ]:
# Create src __init__.py
init_code = '''"""Agentic Logistics Control System."""
from pathlib import Path

__version__ = "0.1.0"
PROJECT_ROOT = Path(__file__).parent.parent
'''

src_init = PROJECT_ROOT / "src" / "__init__.py"
src_init.write_text(init_code)
print(f"Created {src_init}")

## 6. Verification

In [ ]:
# Final verification
print("=" * 50)
print("PROJECT SETUP VERIFICATION")
print("=" * 50)

checks = {
    "Python >= 3.10": sys.version_info >= (3, 10),
    "LangGraph installed": "langgraph" in sys.modules or __import__("langgraph"),
    "SymPy installed": "sympy" in sys.modules or __import__("sympy"),
    "Pydantic installed": "pydantic" in sys.modules or __import__("pydantic"),
    "Config loaded": settings is not None,
    "Project structure valid": all((PROJECT_ROOT / d).exists() for d in expected_dirs),
}

all_passed = True
for check, passed in checks.items():
    status = "✓" if passed else "✗"
    print(f"  {status} {check}")
    all_passed = all_passed and passed

print("=" * 50)
if all_passed:
    print("✓ All checks passed! Ready for Phase 1 development.")
else:
    print("✗ Some checks failed. Please review and fix issues above.")

## Next Steps

1. Configure `.env` file with your xAI API key
2. Proceed to `02_data_models.ipynb` to define data structures
3. Proceed to `03_perception_layer.ipynb` for the observation layer